In [29]:
#여기 코드가 가장 중요하다. 위의 코드를 사용하면 matplot창을 띄워주고
# 아래의 코드를 사용하면 파이썬에서 첫번째 이미지만 보여준다.


%matplotlib qt

#%matplotlib inline

In [2]:
import tensorflow as tf
import os

def download_model(model_type):
    server_prefix = 'https://omnomnom.vision.rwth-aachen.de/data/metrabs'
    model_zippath = tf.keras.utils.get_file(
        origin=f'{server_prefix}/{model_type}_20211019.zip',
        extract=True, cache_subdir='models')
    model_path = os.path.join(os.path.dirname(model_zippath), model_type)
    return model_path

model = tf.saved_model.load(download_model('metrabs_mob3l_y4t')) # or metrabs_eff2l_y4 for the big model
! wget -q https://raw.githubusercontent.com/isarandi/metrabs/master/img/test_image_3dpw.jpg


'wget'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [37]:
import cv2
image1 = cv2.imread('C:/Users/Leeyourack/jookyoung/image/2.screenshot/3scr239.jpg')
image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
pred1 = model.detect_poses(image1, skeleton='smpl_24')

In [9]:
import cv2
image2 = cv2.imread('C:/Users/Leeyourack/jookyoung/image/2.screenshot/3scr253.jpg')
image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)
pred2 = model.detect_poses(image2, skeleton='smpl_24')

In [10]:
import cv2
image3 = cv2.imread('C:/Users/Leeyourack/jookyoung/image/2.screenshot/3scr299.jpg')
image3 = cv2.cvtColor(image3, cv2.COLOR_BGR2RGB)
pred3 = model.detect_poses(image3, skeleton='smpl_24')

### normalvector

In [11]:
poses3d1=pred1['poses3d'].numpy()[0]
poses3d1[..., 1], poses3d1[..., 2] = poses3d1[..., 2], -poses3d1[..., 1]

In [12]:
a=poses3d1[10]
a

array([-1029. ,  7466.4,   857.1], dtype=float32)

In [13]:
poses3d2=pred2['poses3d'].numpy()[0]
poses3d2[..., 1], poses3d2[..., 2] = poses3d2[..., 2], -poses3d2[..., 1]

In [14]:
b=poses3d2[10]
b

array([-1511.2,  5996.8,   543.6], dtype=float32)

In [15]:
poses3d3=pred3['poses3d'].numpy()[0]
poses3d3[..., 1], poses3d3[..., 2] = poses3d3[..., 2], -poses3d3[..., 1]
 

In [16]:
c=poses3d3[10]
c

array([-181.325, 6224.   , -305.5  ], dtype=float32)

In [17]:
ab = b-a
print(ab)
ac = c-a
print(ac)

[ -482.19995 -1469.6001   -313.5    ]
[  847.675  -1242.3999 -1162.6   ]


In [18]:
list = [ab[1]*ac[2]-ab[2]*ac[1],-(ab[0]*ac[2]-ab[2]*ac[0]),ab[0]*ac[1]-ab[1]*ac[0]]
list

[1319064.6, -826351.75, 1844828.5]

In [19]:
list = [(ab[1]*ac[2]-ab[2]*ac[1])/100000,
        -(ab[0]*ac[2]-ab[2]*ac[0])/100000,
            (ab[0]*ac[1]-ab[1]*ac[0])/100000]
list

[13.19064625, -8.2635175, 18.448285]

In [20]:
list[0]

13.19064625

### rotation matrix

In [21]:
x=list[0]
y=list[1]
z=list[2]
print('x:',x, 'y:',y, 'z:',z )
xy = (x**2+y**2)**0.5
yz = (y**2+z**2)**0.5
xz = (x**2+z**2)**0.5
print('xy:',xy, 'yz:',yz, 'xz:',xz )

xyz = (x**2+y**2+z**2)**0.5
print('xyz:',xyz)


x: 13.19064625 y: -8.2635175 z: 18.448285
xy: 15.565309825552632 yz: 20.214473550256788 xz: 22.67889697348317
xyz: 24.1374830793658


In [22]:
#z축 회전

import math
cos1 = math.cos(x/xy)
sin1 = math.sin(x/xy)

print(cos1,sin1)

0.6619052177385927 0.7495875417390728


In [23]:
#y축 회전
import math
cos2 = math.cos(xy/xyz)
sin2 = math.sin(xy/xyz)
sin2 = -sin2

print(cos2,sin2)

0.7991836159339522 -0.6010869720953311


In [61]:
import numpy as np

r1 = np.array([[cos1,-sin1,0], #z축 회전
                [sin1,cos1,0],
                [0,0,1]])
r2 = np.array([[cos2,0,sin2],
                [0,1,0],
                [-sin2,0,cos2]])        #y축 회전
r3 = np.array([[0,1,0], #z축 회전
                [-1,0,0],
                [0,0,1]])
                

In [62]:
rotation1 = r3@r2@r1
rotation1

array([[ 0.74958754,  0.66190522,  0.        ],
       [-0.52898381,  0.59905808,  0.60108697],
       [ 0.3978626 , -0.45056731,  0.79918362]])

In [73]:
# 변환적용 전

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from mpl_toolkits.mplot3d import Axes3D
import numpy as np 

def visualize(im, detections, poses3d, poses2d, edges):
    fig = plt.figure(figsize=(10, 5.2))
    image_ax = fig.add_subplot(1, 2, 1)
    image_ax.imshow(image1)
    for x, y, w, h in detections[:, :4]:
        image_ax.add_patch(Rectangle((x, y), w, h, fill=False))
    

    pose_ax = fig.add_subplot(1, 2, 2, projection='3d')
    pose_ax.view_init(14, -76)
    pose_ax.set_xlim3d(-10000, 1500)
    pose_ax.set_zlim3d(0, 6000)
    pose_ax.set_ylim3d(0, 20000)


    # x, y, z값 
    xpos = [-10000,-1500,500]
    ypos = [10000,9000,5000]
    zpos = [0,0,0]

    # delta x, y, z 값
    dx = [4000,2000,1000]
    dy = [10000,11000,15000]
    dz = [2000,2000,2000]

    #그래프 만들기
    pose_ax.bar3d(xpos,ypos,zpos,dx,dy,dz,'#7f7f7f',alpha=0.2) #,color='#00ceaa'(초록색) 기본값은 파란색
    pose_ax.set_title('6F')
    pose_ax.set_xlabel('x axis')
    pose_ax.set_ylabel('y axis')
    pose_ax.set_zlabel('z axis')

    '''#2
    pose_ax = fig.add_subplot(1, 2, 2, projection='3d')
    pose_ax.view_init(5, -85)
    pose_ax.set_xlim3d(-1500, 1500)
    pose_ax.set_zlim3d(-500, 1500)
    pose_ax.set_ylim3d(0, 3000)'''



    # Matplotlib plots the Z axis as vertical, but our poses have Y as the vertical axis.
    # Therefore, we do a 90° rotation around the X axis:
    poses3d[..., 1], poses3d[..., 2] = poses3d[..., 2], -poses3d[..., 1] 
    '''a=np.ones(24)*(-6000)
    b=np.ones(24)
    c=np.ones(24)*(-2700)
    shift=np.row_stack((a,b,c))'''
    

    for pose3d, pose2d in zip(poses3d, poses2d):
        #p1=pose3d.T+shift
        #pose3d=rotation1@pose3d.T
        #pose3d=pose3d+shift
        for i_start, i_end in edges:
            image_ax.plot(*zip(pose2d[i_start], pose2d[i_end]), marker='o', markersize=2)
            pose_ax.plot(*zip(pose3d[i_start], pose3d[i_end]), marker='o', markersize=2)
        image_ax.scatter(*pose2d.T, s=2)
        pose_ax.scatter(*pose3d.T, s=2)

    fig.tight_layout()
    plt.show()

visualize(
    image1, 
    pred1['boxes'].numpy(),
    pred1['poses3d'].numpy(),
    pred1['poses2d'].numpy(),
    model.per_skeleton_joint_edges['smpl_24'].numpy())

In [69]:
##1
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from mpl_toolkits.mplot3d import Axes3D
import numpy as np 

def visualize(im, detections, poses3d, poses2d, edges):
    fig = plt.figure(figsize=(10, 5.2))
    image_ax = fig.add_subplot(1, 2, 1)
    image_ax.imshow(image1)
    for x, y, w, h in detections[:, :4]:
        image_ax.add_patch(Rectangle((x, y), w, h, fill=False))
    

    pose_ax = fig.add_subplot(1, 2, 2, projection='3d')
    pose_ax.view_init(14, -76)
    pose_ax.set_xlim3d(-10000, 1500)
    pose_ax.set_zlim3d(0, 6000)
    pose_ax.set_ylim3d(0, 20000)


    # x, y, z값 
    xpos = [-10000,-1500,500]
    ypos = [10000,9000,5000]
    zpos = [0,0,0]

    # delta x, y, z 값
    dx = [4000,2000,1000]
    dy = [10000,11000,15000]
    dz = [2000,2000,2000]

    #그래프 만들기
    pose_ax.bar3d(xpos,ypos,zpos,dx,dy,dz,'#7f7f7f',alpha=0.2) #,color='#00ceaa'(초록색) 기본값은 파란색
    pose_ax.set_title('6F')
    pose_ax.set_xlabel('x axis')
    pose_ax.set_ylabel('y axis')
    pose_ax.set_zlabel('z axis')

    '''#2
    pose_ax = fig.add_subplot(1, 2, 2, projection='3d')
    pose_ax.view_init(5, -85)
    pose_ax.set_xlim3d(-1500, 1500)
    pose_ax.set_zlim3d(-500, 1500)
    pose_ax.set_ylim3d(0, 3000)'''



    # Matplotlib plots the Z axis as vertical, but our poses have Y as the vertical axis.
    # Therefore, we do a 90° rotation around the X axis:
    poses3d[..., 1], poses3d[..., 2] = poses3d[..., 2], -poses3d[..., 1] 
    a=np.ones(24)*(-11000)
    b=np.ones(24)*(+4000)
    c=np.ones(24)*(+2500)
    shift=np.row_stack((a,b,c))
    

    for pose3d, pose2d in zip(poses3d, poses2d):
        
        pose3d=rotation1@pose3d.T
        pose3d=pose3d+shift

        for i_start, i_end in edges:
            image_ax.plot(*zip(pose2d[i_start], pose2d[i_end]), marker='o', markersize=2)
            pose_ax.plot(*zip(pose3d.T[i_start], pose3d.T[i_end]), marker='o', markersize=2)
        
        image_ax.scatter(*pose2d.T, s=2)
        pose_ax.scatter(*pose3d, s=2)

    fig.tight_layout()
    plt.show()

visualize(
    image1, 
    pred1['boxes'].numpy(),
    pred1['poses3d'].numpy(),
    pred1['poses2d'].numpy(),
    model.per_skeleton_joint_edges['smpl_24'].numpy())

In [71]:
##2
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from mpl_toolkits.mplot3d import Axes3D
import numpy as np 

def visualize(im, detections, poses3d, poses2d, edges):
    fig = plt.figure(figsize=(10, 5.2))
    image_ax = fig.add_subplot(1, 2, 1)
    image_ax.imshow(image2)
    for x, y, w, h in detections[:, :4]:
        image_ax.add_patch(Rectangle((x, y), w, h, fill=False))
    

    pose_ax = fig.add_subplot(1, 2, 2, projection='3d')
    pose_ax.view_init(14, -76)
    pose_ax.set_xlim3d(-10000, 1500)
    pose_ax.set_zlim3d(0, 6000)
    pose_ax.set_ylim3d(0, 20000)


    # x, y, z값 
    xpos = [-10000,-1500,500]
    ypos = [10000,9000,5000]
    zpos = [0,0,0]

    # delta x, y, z 값
    dx = [4000,2000,1000]
    dy = [10000,11000,15000]
    dz = [2000,2000,2000]

    #그래프 만들기
    pose_ax.bar3d(xpos,ypos,zpos,dx,dy,dz,'#7f7f7f',alpha=0.2) #,color='#00ceaa'(초록색) 기본값은 파란색
    pose_ax.set_title('6F')
    pose_ax.set_xlabel('x axis')
    pose_ax.set_ylabel('y axis')
    pose_ax.set_zlabel('z axis')

    '''#2
    pose_ax = fig.add_subplot(1, 2, 2, projection='3d')
    pose_ax.view_init(5, -85)
    pose_ax.set_xlim3d(-1500, 1500)
    pose_ax.set_zlim3d(-500, 1500)
    pose_ax.set_ylim3d(0, 3000)'''



    # Matplotlib plots the Z axis as vertical, but our poses have Y as the vertical axis.
    # Therefore, we do a 90° rotation around the X axis:
    poses3d[..., 1], poses3d[..., 2] = poses3d[..., 2], -poses3d[..., 1] 
    a=np.ones(24)*(-11000)
    b=np.ones(24)*(+4000)
    c=np.ones(24)*(+2500)
    shift=np.row_stack((a,b,c))
    

    for pose3d, pose2d in zip(poses3d, poses2d):
        
        pose3d=rotation1@pose3d.T
        pose3d=pose3d+shift

        for i_start, i_end in edges:
            image_ax.plot(*zip(pose2d[i_start], pose2d[i_end]), marker='o', markersize=2)
            pose_ax.plot(*zip(pose3d.T[i_start], pose3d.T[i_end]), marker='o', markersize=2)
        
        image_ax.scatter(*pose2d.T, s=2)
        pose_ax.scatter(*pose3d, s=2)

    fig.tight_layout()
    plt.show()

visualize(
    image2, 
    pred2['boxes'].numpy(),
    pred2['poses3d'].numpy(),
    pred2['poses2d'].numpy(),
    model.per_skeleton_joint_edges['smpl_24'].numpy())

In [72]:
##3
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from mpl_toolkits.mplot3d import Axes3D
import numpy as np 

def visualize(im, detections, poses3d, poses2d, edges):
    fig = plt.figure(figsize=(10, 5.2))
    image_ax = fig.add_subplot(1, 2, 1)
    image_ax.imshow(image3)
    for x, y, w, h in detections[:, :4]:
        image_ax.add_patch(Rectangle((x, y), w, h, fill=False))
    

    pose_ax = fig.add_subplot(1, 2, 2, projection='3d')
    pose_ax.view_init(14, -76)
    pose_ax.set_xlim3d(-10000, 1500)
    pose_ax.set_zlim3d(0, 6000)
    pose_ax.set_ylim3d(0, 20000)


    # x, y, z값 
    xpos = [-10000,-1500,500]
    ypos = [10000,9000,5000]
    zpos = [0,0,0]

    # delta x, y, z 값
    dx = [4000,2000,1000]
    dy = [10000,11000,15000]
    dz = [2000,2000,2000]

    #그래프 만들기
    pose_ax.bar3d(xpos,ypos,zpos,dx,dy,dz,'#7f7f7f',alpha=0.2) #,color='#00ceaa'(초록색) 기본값은 파란색
    pose_ax.set_title('6F')
    pose_ax.set_xlabel('x axis')
    pose_ax.set_ylabel('y axis')
    pose_ax.set_zlabel('z axis')

    '''#2
    pose_ax = fig.add_subplot(1, 2, 2, projection='3d')
    pose_ax.view_init(5, -85)
    pose_ax.set_xlim3d(-1500, 1500)
    pose_ax.set_zlim3d(-500, 1500)
    pose_ax.set_ylim3d(0, 3000)'''



    # Matplotlib plots the Z axis as vertical, but our poses have Y as the vertical axis.
    # Therefore, we do a 90° rotation around the X axis:
    poses3d[..., 1], poses3d[..., 2] = poses3d[..., 2], -poses3d[..., 1] 
    a=np.ones(24)*(-11000)
    b=np.ones(24)*(+4000)
    c=np.ones(24)*(+2500)
    shift=np.row_stack((a,b,c))
    

    for pose3d, pose2d in zip(poses3d, poses2d):
        
        pose3d=rotation1@pose3d.T
        pose3d=pose3d+shift
        for i_start, i_end in edges:
            image_ax.plot(*zip(pose2d[i_start], pose2d[i_end]), marker='o', markersize=2)
            pose_ax.plot(*zip(pose3d.T[i_start], pose3d.T[i_end]), marker='o', markersize=2)
        image_ax.scatter(*pose2d.T, s=2)
        pose_ax.scatter(*pose3d, s=2)

    fig.tight_layout()
    plt.show()

visualize(
    image3, 
    pred3['boxes'].numpy(),
    pred3['poses3d'].numpy(),
    pred3['poses2d'].numpy(),
    model.per_skeleton_joint_edges['smpl_24'].numpy())